In [35]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [37]:
# =======================
# 0. Definir ruta base del proyecto
# =======================
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
ruta_gold = os.path.join(ruta_base, "GOLD")

In [43]:
# --- 1. Cargar el dataset con características desde la capa GOLD ---
df_modelo = pd.read_parquet(os.path.join(ruta_gold, "maloob_features_for_training.parquet"))

In [45]:
# --- 3. Definir las nuevas Características (X) y el Objetivo (y) ---
# Incluimos TODAS las nuevas "pistas" que creamos.
features = [
        'produccion_lag_1',
        'produccion_lag_6',
        'produccion_lag_12',
        'produccion_roll_mean_6',
        'produccion_roll_std_6',
        'mes',
        'trimestre',
        'año'
    ]
target = 'petroleo_mbd'

X = df_modelo[features]
y = df_modelo[target]
print("\nCaracterísticas (X) y objetivo (y) definidos.")


Características (X) y objetivo (y) definidos.


In [47]:
# --- 4. Dividir los datos en Entrenamiento y Prueba ---
# Mantenemos la misma estrategia de división para una comparación justa.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Datos divididos: {len(X_train)} filas para entrenamiento, {len(X_test)} filas para prueba.")

# --- 5. Entrenar el Modelo XGBoost ---
# Usamos la misma configuración del modelo para ver el impacto REAL de las nuevas características.
model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        early_stopping_rounds=5
    )
    
model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
print("Modelo re-entrenado con éxito usando las nuevas características.")

Datos divididos: 77 filas para entrenamiento, 20 filas para prueba.
Modelo re-entrenado con éxito usando las nuevas características.


In [49]:
# --- 6. Evaluar el Modelo Mejorado ---
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
print("\n--- ¡RESULTADO DEL MODELO MEJORADO! ---")
print(f"RMSE Anterior: 106.31")
print(f"RMSE Nuevo: {rmse:.2f}")

if rmse < 106.31:
        print("\n¡Felicitaciones! El error se redujo. La nueva ingeniería de características fue un éxito.")
else:
        print("\nEl error no se redujo. Es un resultado de aprendizaje valioso, podemos analizar por qué.")


--- ¡RESULTADO DEL MODELO MEJORADO! ---
RMSE Anterior: 106.31
RMSE Nuevo: 7.45

¡Felicitaciones! El error se redujo. La nueva ingeniería de características fue un éxito.
